
# Data Science Part Time Course - Sydney 
## Natural Language Processing (NLP)

This notebook contains exercises for getting started with Natual Language Processing in Python. The main topics we will cover in this class are:

Introduction
1. newsgroups dataset
- Bag of words prediciton model

Advanced Language Processing with NLTK 
1. Tokenizing
- Stemming
- Speech Tagging
- Named Entity Recognition
- Term Frequency - Inverse Document Frequency
- Latent Dirichlet Allocation
- Regex

## Bag of Words Prediction

We will use the [20 Newsgroup dataset](http://qwone.com/~jason/20Newsgroups/), which is provided by Scikit-Learn. This is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering.

We will restrict the analysis to 4 groups and will attempt to classify them starting from the corresponding text.

This is a typical example of text classification, where a data scientist's task is to train a model that can partition text in pre-defined categories. Other examples include sentiment analysis and topic assignment.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
# 4 (of 20 available) categories of news articles to download
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

# retrieve the prepared train data
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

# retrieve the prepared test data
data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=('headers', 'footers', 'quotes'))

## 1. Data inspection

We have downloaded a few newsgroup categories and removed headers, footers and quotes.

Let's inspect them.


Q: What data type is `data_train` ?

Q: how to you retrieve the data?

In [ ]:
# use the .data method to retrive the data component form the train and test object


Q: Inspect the first data point, what does it look like?

## 2. Bag of Words model

Let's train a model using a simple count vectorizer

What is CountVectoriser? 
See the following documentation. It is for counting occurances of words.

[Text Feature Extraction](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

Workflow we will follow:

1. Initialize a standard CountVectorizer and fit the training data (tokenization and occurrence counting)
- how big is the feature dictionary
- repeat eliminating english stop words
- is the dictionary smaller?
- transform the training data using the trained vectorizer
- what are the 20 words that are most common in the whole corpus?
- what are the 20 most common words in each of the 4 classes?
- evaluate the performance of a Logistic Regression on the features extracted by the CountVectorizer
    - you will have to transform the test_set too. Be carefule to use the trained vectorizer, without re-fitting it
- try the following 3 modification:
    - restrict the max_features
    - change max_df and min_df
    - use a fixed vocabulary of size 80 combining the 20 most common words per group found earlier
- for each of the above print a confusion matrix and investigate what gets mixed
- print out the number of features for each model

In [ ]:
# Initialize a standard CountVectorizer and fit the training data (tokenization and occurrence counting)
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# intialise CountVectorizer and fit on the dataset
cvec = CountVectorizer()
cvec.fit(data_train['data'])

Q: how big is the feature dictionary

In [ ]:
# repeat the fit process, this time eliminating english stop words
cvec = CountVectorizer(stop_words='english')
cvec.fit(data_train['data'])
len(cvec.get_feature_names())

Q: is the dictionary smaller?

In [ ]:
# transform the training data using the trained vectorizer
X_train = pd.DataFrame(cvec.transform(data_train['data']).todense(),
                       columns=cvec.get_feature_names())

In [ ]:
y_train = data_train['target']

Q: what are the 20 words that are most common in the whole corpus?

In [ ]:
word_counts = X_train.sum(axis=0)
word_counts.sort_values(ascending = False).head(20)

In [ ]:
# 4 target names
names = data_train['target_names']
names

In [ ]:
# what are the 20 most common words in each of the 4 classes?

common_words = []
for i in range(4):
    word_count = X_train[y_train==i].sum(axis=0)
    print(names[i], "most common words")
    cw = word_count.sort_values(ascending = False).head(20)
#     cw.to_csv('../../../5.2-lesson/assets/datasets/'+names[i]+'_most_common_words.csv')
    print(cw)
    common_words.extend(cw.index)
    print()

Evaluate the performance of a Logistic Regression on the features extracted by the CountVectorizer

In [ ]:
X_test = pd.DataFrame(cvec.transform(data_test['data']).todense(),
                      columns=cvec.get_feature_names())

In [ ]:
X_test = pd.DataFrame(cvec.transform(data_test['data']).todense(),
                      columns=cvec.get_feature_names())
y_test = data_test['target']

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

try the following 3 modification:

- restrict the max_features
- change max_df and min_df
- use a fixed vocabulary of size 80 combining the 20 most common words per group found earlier

Restrict the max_features to 1000:

In [ ]:
?CountVectorizer

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(CountVectorizer(stop_words='english',
                                      max_features=________),
                      LogisticRegression(),
                      )
model.fit(data_train['data'], y_train)
y_pred = model.predict(data_test['data'])
print(accuracy_score(y_test, y_pred))
docm(y_test, y_pred, names)
print("Number of features:", len(model.steps[0][1].get_feature_names()))

Print a confusion matrix and investigate what gets mixed

In [ ]:
def docm(y_true, y_pred, labels=None):
    cm = confusion_matrix(y_true, y_pred)
    if labels is not None:
        cols = ['p_'+c for c in labels]
        df = pd.DataFrame(cm, index=labels, columns=cols)
    else:
        cols = ['p_'+str(i) for i in xrange(len(cm))]
        df = pd.DataFrame(cm, columns=cols)
    return df

In [ ]:
docm(y_test, y_pred, names)

Change max_features to 2000 and min_df to 0.01 and max_df to 0.95:

In [ ]:
model = make_pipeline(CountVectorizer(stop_words='english',
                                      max_features=_____,
                                      min_df=______,
                                      max_df=______),
                      LogisticRegression(),
                      )
model.fit(data_train['data'], y_train)
y_pred = model.predict(data_test['data'])
print(accuracy_score(y_test, y_pred))
docm(y_test, y_pred, names)
print("Number of features:", len(model.steps[0][1].get_feature_names()))

In [ ]:
docm(y_test, y_pred, names)

Use a fixed vocabulary of size 80 combining the 20 most common words per group found earlier

In [ ]:
model = make_pipeline(CountVectorizer(stop_words='english',
                                      vocabulary=set(common_words)),
                      LogisticRegression(),
                      )
model.fit(data_train['data'], y_train)
y_pred = model.predict(data_test['data'])
print(accuracy_score(y_test, y_pred))
docm(y_test, y_pred, names)
print("Number of features:", len(model.steps[0][1].get_feature_names()))

In [ ]:
docm(y_test, y_pred, names)

# Natural Language Processing with NLTK

In [ ]:
import nltk

If you have not downloaded NLTK, use the following method to download just the essentials:

For the following step, see the new window that will pop out elsewhere. The whole NLTK package is huge. Too big for all of you to download in class at the same time. Instead, select to install just the main packages and see of the lab runs. If not, use the downloader to get the additional missing packages.

In [ ]:
# find pop-up window for downloader tool
#nltk.download()

### Tokenization

What:  Separate text into units such as sentences or words

Why:   Gives structure to previously unstructured text

Notes: Relatively easy with English language text, not easy with some languages


"corpus" = collection of documents

"corpora" = plural form of corpus


In [ ]:
import requests
from bs4 import BeautifulSoup

r = requests.get("http://en.wikipedia.org/wiki/Data_science")
b = BeautifulSoup(r.text, "lxml")
paragraphs = b.find("body").findAll("p")
text = ""
for paragraph in paragraphs:
    text += paragraph.text + " "
# Data Science corpus
text[:500]

In [ ]:
# tokenize into sentences
sentences = [sent for sent in nltk.sent_tokenize(text)]
sentences[:10]


In [ ]:
# tokenize into words
tokens = [word for word in nltk.word_tokenize(text)]
tokens[:100]

In [ ]:
# only keep tokens that start with a letter (using regular expressions)
import re
clean_tokens = [token for token in tokens if re.search('^[a-zA-Z]+', token)]
clean_tokens[:100]


In [ ]:
# count the tokens
from collections import Counter
c = Counter(clean_tokens)

c.most_common(25)       # mixed case

In [ ]:
sorted(c.items())[:25]  # counts similar words separately

#print
for item in sorted(c.items())[:25]:
    print(item[0], item[1])

###################
##### EXERCISE ####
###################

- Put each word in clean_tokens in lower case
- find the new word count of the lowered tokens
- Then show the top 10 words used in this corpus


In [ ]:
# Put each word in clean_tokens in lower case


In [ ]:
# find the new word count of the lowered tokens


In [ ]:
# Then show the top 10 words used in this corpus


### Stemming
What:  Reduce a word to its base/stem form

Why:   Often makes sense to treat multiple word forms the same way

Notes: 

- Uses a "simple" and fast rule-based approach
- Output can be undesirable for irregular words
- Stemmed words are usually not shown to users (used for analysis/indexing)
- Some search engines treat words with the same stem as synonyms

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')


In [ ]:
# example stemming
stemmer.stem('charge')
stemmer.stem('charging')
stemmer.stem('charged')

In [ ]:
# stem the tokens
stemmed_tokens = [stemmer.stem(t) for t in clean_tokens]


In [ ]:
# count the stemmed tokens
c = Counter(stemmed_tokens)
c.most_common(25)       # all lowercase
sorted(c.items())[:25]  # some are strange

### Lemmatization
What:  Derive the canonical form ('lemma') of a word
    
Why:   Can be better than stemming, reduces words to a 'normal' form.
    
Notes: Uses a dictionary-based approach (slower than stemming)
    

In [ ]:
lemmatizer = nltk.WordNetLemmatizer()

##### compare stemmer to lemmatizer

'dogs'

In [ ]:
# compare stemmer to lemmatizer
stemmer.stem('dogs')

In [ ]:
lemmatizer.lemmatize('dogs')

'wolves'

In [ ]:
stemmer.stem('wolves') # Beter for information retrieval and search

In [ ]:
lemmatizer.lemmatize('wolves') # Better for text analysis

'is'

In [ ]:
stemmer.stem('is')

In [ ]:
lemmatizer.lemmatize('is')

In [ ]:
lemmatizer.lemmatize('is',pos='v')

### Part of Speech Tagging

What:  Determine the part of speech of a word
    
Why:   This can inform other methods and models such as Named Entity Recognition
    
Notes: http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [ ]:
temp_sent = 'We have a revision class this Saturday'
# pos_tag takes a tokenize sentence
nltk.pos_tag(nltk.word_tokenize(temp_sent))

### Stopword Removal

What:  Remove common words that will likely appear in any text
    
Why:   They don't tell you much about your text

In [ ]:
# most of top 25 stemmed tokens are "worthless"
c.most_common(25)

In [ ]:
# view the list of stopwords
stopwords = nltk.corpus.stopwords.words('english')
sorted(stopwords)

##################
### Exercise  ####
##################


- Create a variable called stemmed_stops which is the stemmed version of each stopword in stopwords. Use the stemmer we used up above!
- Then create a list called stemmed_tokens_no_stop that contains only the tokens in stemmed_tokens that aren't in stemmed_stops
- Show the 25 most common stemmed non stop word tokens

In [ ]:
# Create a variable called stemmed_stops which is the stemmed version of each stopword in stopwords


In [ ]:
# create a list called stemmed_tokens_no_stop that contains only the tokens in stemmed_tokens that aren't in stemmed_stops


In [ ]:
# Show the 25 most common stemmed non stop word tokens


### Named Entity Recognition

What:  Automatically extract the names of people, places, organizations, etc.

Why:   Can help you to identify "important" words

Notes: 

- Training NER classifier requires a lot of annotated training data
- Should be trained on data relevant to your task
- Stanford NER classifier is pretty good

In [ ]:
sentence = 'Alasdair is an instructor for General Assembly'

tokenized = nltk.word_tokenize(sentence)

tokenized

In [ ]:
tagged = nltk.pos_tag(tokenized)

tagged


In [ ]:
chunks = nltk.ne_chunk(tagged)

chunks


In [ ]:
def extract_entities(text):
    entities = []
    # tokenize into sentences
    for sentence in nltk.sent_tokenize(text):
        # tokenize sentences into words
        # add part-of-speech tags
        # use NLTK's NER classifier
        chunks = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentence)))
        # parse the results
        entities.extend([chunk for chunk in chunks if hasattr(chunk, 'label')])
    return entities

for entity in extract_entities('Alasdair is an instructor for General Assembly'):
    print('[' + entity.label() + '] ' + ' '.join(c[0] for c in entity.leaves()))

### Term Frequency - Inverse Document Frequency (TF-IDF)

What:  Computes "relative frequency" that a word appears in a document
           compared to its frequency across all documents

Why:   More useful than "term frequency" for identifying "important" words in
           each document (high frequency in that document, low frequency in
           other documents)

Notes: Used for search engine scoring, text summarization, document clustering

How: 
    - TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
    - IDF(t) = log_e(Total number of documents / Number of documents with term t in it).

In [ ]:
sample = ['Bob likes sports', 'Bob hates sports', 'Bob likes likes trees']

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()


In [ ]:
# Each row represents a sentence
# Each column represents a word
vect.fit_transform(sample).toarray()
vect.get_feature_names()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit_transform(sample).toarray()
tfidf.get_feature_names()


In [ ]:
# the IDF of each word
idf = tfidf.idf_
print(dict(list(zip(tfidf.get_feature_names(), idf))))

###############
#### Exercise #####
###############


for each sentence in sample, find the most "interesting words" by ordering their tfidf in ascending order


## Extra Time Demonstrations 

### LDA - Latent Dirichlet Allocation

What:  Way of automatically discovering topics from sentences

Why:   Much quicker than manually creating and identifying topic clusters

In [ ]:
!pip install lda

In [ ]:
sentences[1:5]

In [ ]:
import lda

# Instantiate a count vectorizer with two additional parameters
vect = CountVectorizer(stop_words='english', ngram_range=[1,3]) 
sentences_train = vect.fit_transform(sentences)


In [ ]:
# Instantiate an LDA model
model = lda.LDA(n_topics=10, n_iter=500)
model.fit(sentences_train) # Fit the model 
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vect.get_feature_names())[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ', '.join(topic_words)))


In [ ]:
# EXAMPLE: Automatically summarize a document


# corpus of 2000 movie reviews
from nltk.corpus import movie_reviews
reviews = [movie_reviews.raw(filename) for filename in movie_reviews.fileids()]


In [ ]:
reviews[1:5]

In [ ]:
# create document-term matrix
tfidf = TfidfVectorizer(stop_words='english')
dtm = tfidf.fit_transform(reviews)
features = tfidf.get_feature_names()

In [ ]:
import numpy as np


In [ ]:
# find the most and least "interesting" sentences in a randomly selected review
def summarize():
    
    # choose a random movie review    
    review_id = np.random.randint(0, len(reviews))
    review_text = reviews[review_id]

    # we are going to score each sentence in the review for "interesting-ness"
    sent_scores = []
    # tokenize document into sentences
    for sentence in nltk.sent_tokenize(review_text):
        # exclude short sentences
        if len(sentence) > 6:
            score = 0
            token_count = 0
            # tokenize sentence into words
            tokens = nltk.word_tokenize(sentence)
            # compute sentence "score" by summing TFIDF for each word
            for token in tokens:
                if token in features:
                    score += dtm[review_id, features.index(token)]
                    token_count += 1
            # divide score by number of tokens
            sent_scores.append((score / float(token_count + 1), sentence))

    # lowest scoring sentences
    print('\nLOWEST:\n')
    for sent_score in sorted(sent_scores)[:3]:
        print(sent_score[1])

    # highest scoring sentences
    print('\nHIGHEST:\n')
    for sent_score in sorted(sent_scores, reverse=True)[:3]:
        print(sent_score[1])

# try it out!
summarize()

### TextBlob Demo

In [ ]:
# TextBlob Demo: "Simplified Text Processing"
# Installation: pip install textblob
! pip install textblob

In [ ]:
from textblob import TextBlob, Word

In [ ]:
# identify words and noun phrases
blob = TextBlob('Liam and Sinan are instructors for General Assembly')
blob.words
blob.noun_phrases

In [ ]:
# sentiment analysis
blob = TextBlob('I hate this horrible movie. This movie is not very good.')
blob.sentences
blob.sentiment.polarity
[sent.sentiment.polarity for sent in blob.sentences]

In [ ]:
# sentiment subjectivity
TextBlob("I am a cool person").sentiment.subjectivity # Pretty subjective
TextBlob("I am a person").sentiment.subjectivity # Pretty objective
# different scores for essentially the same sentence
print(TextBlob('Ian and Alasdair are instructors for General Assembly in Sydney').sentiment.subjectivity)

In [ ]:
# singularize and pluralize
blob = TextBlob('Put away the dishes.')
[word.singularize() for word in blob.words]

In [ ]:
[word.pluralize() for word in blob.words]


In [ ]:
# spelling correction
blob = TextBlob('15 minuets late')
blob.correct()

In [ ]:
# spellcheck
Word('parot').spellcheck()


In [ ]:
# definitions
Word('bank').define()
Word('bank').define('v')

In [ ]:
# translation and language identification
blob = TextBlob('Welcome to the classroom.')
blob.translate(to='es')
blob = TextBlob('Hola amigos')
blob.detect_language()

### Regular Expressions - Regex - References

This is the python module for regular expressions: https://docs.python.org/2/library/re.html

Here is a google page for explaining regular expression patterns: https://developers.google.com/edu/python/regular-expressions

And here is a convenient tool for testing regular expressions: https://regex101.com/#python

Have a read of these and play around with regular expressions below and in the regex101 tool